In [1]:
require "awesome_print"

true

In [3]:
class Sanitizer
  def self.sanitize(values)
    return values
  end
end

:sanitize

In [4]:
class Users
    
    @@users ||= {}

    def self.get
        @@users.keys
    end
    
    def self.create_or_update(user, password)
        @@users[user] = password
        puts "User \"#{user}\" was created or updated"
    end

    def self.get_user_password(user)
        @@users[user]
    end

end

:get_user_password

In [5]:
class Auth
    
    def self.current_user
        @@current_user
    end

    def self.login(user, password)
        if Users.get_user_password(user) == password 
            @@current_user = user
            puts "Logged in as #{user}"
        else
            "Wrong user/password"
        end
    end

end

:login

In [6]:
class Permissions

  @@Permissions ||= {}

  def self.add(user, client, role)
    @@Permissions[[user, client]] ||= []
    @@Permissions[[user, client]] << role unless @@Permissions[[user, client]].include?(role)
  end

  def self.get
    @@Permissions
  end

  def self.get_roles(client)
    raise "#{Auth.current_user} does not have a defined role for client #{client}" unless @@Permissions[[Auth.current_user, client]] 
    @@Permissions[[Auth.current_user, client]]
  end
  
end

:get_roles

In [7]:
class RolesToPermissions

  @@roles_to_permissions ||= {}

  def self.set(role, entity, operation, fields)
    @@roles_to_permissions[[role, entity]] ||= {}
    @@roles_to_permissions[[role, entity]][operation] = fields
  end
  
  def self.get(roles, entity)
    roles.map {|role| @@roles_to_permissions[[role, entity]] }
  end

end

:get

In [8]:
class PermissionReader

  def self.has_permission?( entity, fields, client, operation )

    roles_for_client = Permissions.get_roles(client) # [:Administrator, ...]
    roles_permissions = RolesToPermissions.get(roles_for_client, entity) # [{:do_create=>[:create], ... }, {:do_create=>[:create], ... }]

    roles_permissions = roles_permissions.reduce({}) do |acc, hash|
      acc.merge(hash) do |key, old_val, new_val|
        (old_val + new_val).uniq
      end
    end

    missing_permissions = fields - roles_permissions[operation]

    raise "Permission denied for #{Auth.current_user} to perform #{operation} on #{entity}
           Missing permissions: #{missing_permissions}" if missing_permissions != []; return

  end

end

:has_permission?

In [9]:
class Audit

  def initialize
    @log = []
  end

  def insert(changed_fields, all_fields)
    @log << {
      name: all_fields[:name].dup.freeze,
      user: Auth.current_user.dup.freeze,
      changed_fields: changed_fields.dup.freeze,
      all_fields: all_fields.dup.freeze
    }
  end

  def select_entries(**fields)
    @log.each do |log_entry|

      print_log_entries(log_entry) if
        fields.all? { |key, value|
        log_entry[:changed_fields][key] == value }

    end; return
  end

  def print
    @log.each do | log_entry |
      print_log_entries(log_entry)
    end; return
  end

  def print_log_entries(log_entry)
    puts "#{log_entry[:user]} changed the following values for #{log_entry[:name]}:"
    ap log_entry[:changed_fields]
    
    puts "New values:"
    ap log_entry[:all_fields]
    puts
  end

  def clear_log
    @log = []
  end

end

:clear_log

In [10]:
module CommonClassMethods

    def validate_data_structure(values)
      new_structure = values.each        { |k, v| values[k] = false if v == true }
      new_structure = values.map         { |k, v| {k => v.class} }
      old_structure = self::Structure.map{ |k, v| {k => v.class} }

      mismatches    = new_structure - old_structure
      raise "Unexpected key or value type: #{mismatches}" if mismatches != []
    end

end

:validate_data_structure

In [11]:
module CommonCreateMethods

    def initialize(values)
        values = Sanitizer.sanitize(values)
        @entity = self.class.name.split('::').last.to_sym # => :Invoice, :SalesOrder etc.
        create(values)
    end

    def create(values)                # ( entity,  fields,   client,          operation )
        PermissionReader.has_permission?(@entity, [:create], values[:client], :do_create )
        
        self.class.validate_data_structure( values )
        self.class.validate_logic(values)

        @values = values
        self.class::Log.insert(values, @values)
    
        puts "#{@entity} was created:"
        ap @values
    end
end

:create

In [12]:
module CommonReadMethod

  def read(fields)
    PermissionReader.has_permission?( @entity, fields, @values[:client], :read )
    @values.slice(*fields)
  end
end

:read

In [13]:
module CommonUpdateMethod

  def update(fields)

  #[1] Define an updated state
    new_values = @values.merge(fields)

  #[2] Check permissions and validation
    PermissionReader.has_permission?( @entity, fields.keys, @values[:client], :update )

    self.class.validate_data_structure(new_values)
    fields = self.class.transition(new_values, fields)
    self.class.validate_logic(new_values)

  #[3] Update entity with new values and write audit
  
    #exclude fields that did not change
    fields = fields.reject { |key, value| @values[key] == value }

    @values.merge!(fields)
    self.class::Log.insert(fields, @values)

  #[4] Print result
    puts "#{@entity} was updated. New values:"
    ap @values

  end
end

:update

In [14]:
module CommonPrintMethod

  def print
    puts "# ---------- #"
    puts
    puts "#{@entity} #{@values[:name]}"
    puts
    @values.each {|key, value| puts "#{key.capitalize}: #{value}" }
    puts
    puts "# ---------- #"
  end

end

:print

In [15]:
module CommonInstanceMethods    
    include CommonCreateMethods
    include CommonReadMethod
    include CommonUpdateMethod
    include CommonPrintMethod
end

#<Class:0x00000220366b6cc0>::CommonInstanceMethods

In [16]:
class SalesOrder
    extend CommonClassMethods
    include CommonInstanceMethods
    
    ClassName ||= self.name.split('::').last.to_sym
    Log ||= Audit.new

    Structure ||= {
      name: "", 
      amount: 0,
      client: "", 
      recognized: false
    }

  # -------------------- #

    def self.validate_logic(new_values)
      return
    end
  
    def self.transition(values, fields)
      fields
    end

    def create_invoice(invoice_name)
      Invoice.new(
        {name: invoice_name, 
         amount: @values[:amount], 
         paid_amount: 0,
         client: @values[:client], 
         is_paid: false}
      )
    end

  end

:create_invoice

In [26]:
class Invoice
  extend CommonClassMethods
  include CommonInstanceMethods
  
  Log ||= Audit.new
 
  Structure ||= {
    name: "", 
    amount: 0,
    paid_amount: 0,
    client: "",
    is_paid: false
  }

  # -------------------- #

  def self.validate_logic(new_values)
    if new_values[:amount] < 100
      raise "money is smol"
    end
  end

  def self.transition(values, fields)
    #is_paid = true/false, based on amount and paid_amount
    values[:amount] <= values[:paid_amount] ?
      fields.merge!(is_paid: true) :
      fields.merge!(is_paid: false)
  end

  def pay(sum)
    paid_amount = @values.slice(:paid_amount)
    paid_amount[:paid_amount] += sum
    update(paid_amount)
  end

end

:pay

In [18]:
Users.create_or_update("Admin", "god")
Users.create_or_update("Kate", "123")
Users.create_or_update("Fluffy", "dog")

Users.get

User "Admin" was created or updated
User "Kate" was created or updated
User "Fluffy" was created or updated


["Admin", "Kate", "Fluffy"]

In [19]:
Permissions.add("Admin", "ASD", :Administrator)
Permissions.add("Kate", "ASD", :InvoicingSpecialist)
Permissions.add("Fluffy", "ASD", :Dog)

[:Dog]

In [20]:
RolesToPermissions.set( :Administrator,       :Invoice,    :do_create,       [:create])
RolesToPermissions.set( :Administrator,       :Invoice,    :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :Invoice,    :update,          [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :Invoice,    :do_delete,       [:delete])

RolesToPermissions.set( :InvoicingSpecialist, :Invoice,    :do_create,       [:create])
RolesToPermissions.set( :InvoicingSpecialist, :Invoice,    :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :InvoicingSpecialist, :Invoice,    :update,          [:amount, :paid_amount, :is_paid])
RolesToPermissions.set( :InvoicingSpecialist,  :Invoice,   :do_delete,       [:delete])

# --- #

RolesToPermissions.set( :Administrator,       :SalesOrder, :do_create,       [:create])
RolesToPermissions.set( :Administrator,       :SalesOrder, :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :SalesOrder, :update,          [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :Administrator,       :SalesOrder, :do_delete,       [:amount, :paid_amount, :client, :name, :is_paid])

RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :do_create,       [:create])
RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :read,            [:amount, :paid_amount, :client, :name, :is_paid])
RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :update,          [:amount, :recognized])
RolesToPermissions.set( :InvoicingSpecialist, :SalesOrder, :do_delete,       [])


[]

In [21]:
Auth.login("Admin", "god")

Logged in as Admin


In [ ]:
Auth.login("Kate", "123")

In [22]:
so1 = SalesOrder.new ( 
    {name: "SO-1", 
    amount: 100,
    client: "ASD", 
    recognized: false} )

SalesOrder was created:
{
          :name => "SO-1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}


#<#<Class:0x00000220366b6cc0>::SalesOrder:0x0000022035df4358 @entity=:SalesOrder, @values={:name=>"SO-1", :amount=>100, :client=>"ASD", :recognized=>false}>

In [23]:
so1 = SalesOrder.new ( 
    {name: "so1", 
    amount: 100,
    client: "ASD", 
    recognized: false} )

SalesOrder was created:
{
          :name => "so1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}


#<#<Class:0x00000220366b6cc0>::SalesOrder:0x0000022036499938 @entity=:SalesOrder, @values={:name=>"so1", :amount=>100, :client=>"ASD", :recognized=>false}>

In [24]:
SalesOrder::Log.print

Admin changed the following values for SO-1:
{
          :name => "SO-1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}
New values:
{
          :name => "SO-1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}

Admin changed the following values for so1:
{
          :name => "so1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}
New values:
{
          :name => "so1",
        :amount => 100,
        :client => "ASD",
    :recognized => false
}



In [27]:
invoice1 = so1.create_invoice("ASD-1")

Invoice was created:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}


#<#<Class:0x00000220366b6cc0>::Invoice:0x000002203661e1a0 @entity=:Invoice, @values={:name=>"ASD-1", :amount=>100, :paid_amount=>0, :client=>"ASD", :is_paid=>false}>

In [28]:
invoice1.update( amount: 120, name: "ASD-3"  )
#invoice1.read([:amount, :is_paid])

Invoice was updated. New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}


In [29]:
invoice1.pay(20)

Invoice was updated. New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 20,
         :client => "ASD",
        :is_paid => false
}


In [30]:
Invoice::Log.print

Admin changed the following values for ASD-1:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}
New values:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}

Admin changed the following values for ASD-3:
{
    :amount => 120,
      :name => "ASD-3"
}
New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}

Admin changed the following values for ASD-3:
{
    :paid_amount => 20
}
New values:
{
           :name => "ASD-3",
         :amount => 120,
    :paid_amount => 20,
         :client => "ASD",
        :is_paid => false
}



In [31]:
Invoice::Log.select_entries(amount: 100, is_paid: false)

Admin changed the following values for ASD-1:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}
New values:
{
           :name => "ASD-1",
         :amount => 100,
    :paid_amount => 0,
         :client => "ASD",
        :is_paid => false
}



In [ ]:
Invoice::Log.clear_log

In [32]:
so1.print

# ---------- #

SalesOrder so1

Name: so1
Amount: 100
Client: ASD
Recognized: false

# ---------- #


In [33]:
invoice1.print

# ---------- #

Invoice ASD-3

Name: ASD-3
Amount: 120
Paid_amount: 20
Client: ASD
Is_paid: false

# ---------- #
